In [1]:
from paho.mqtt import client as mqtt_client
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
import os
import json
from utils import *

import sys
import time
from IPython.display import display, clear_output
import torch
from envs import OptimizationEnv
from sac import SoftActorCritic
from rlutils import ReturnTracker, ReplayBuffer

In [5]:
broker = 'localhost'
port = 10401 # 1883
client_id = 'clip_listener-secondary'
username = 'fwagner'
password = '1234'

In [6]:
def receive_and_print(client, userdata, msg):
    
    try:
        print(time.time(), msg.topic)
        data = json.loads(msg.payload)
        for k,v in zip(data.keys(), data.values()):
            if k == 'Samples':
                pass
            else:
                print(k, v)
        if 'TPA' in data and 'RMS' in data and 'PulseHeight' in data:
            print('Reward: ', - data['RMS'] * np.minimum(data['TPA'], 10.) / np.maximum(data['PulseHeight'], data['RMS']) )
        if 'Samples' in data:
            plt.subplot(1,2,1)
            plt.plot(np.array(data['Samples'])/65536*10)
            plt.subplot(1,2,2)
            fft = np.abs(np.fft.rfft(np.array(data['Samples'])/65536*10))**2
            freq = np.fft.rfftfreq(16384, 1/25000)
            plt.loglog(freq, fft)
            plt.show()
            

    except KeyError as err_msg:
        print('KeyError: ', err_msg)
        pass

In [7]:
client = connect_mqtt(broker, port, client_id, username, password, userdata={})

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
subscribe(client, 'ccscresst/subscription/set')
subscribe(client, 'ccscresst/subscription/ack')
subscribe(client, 'ccscresst/trigger/samples')
subscribe(client, 'ccscresst/control/set')
subscribe(client, 'ccscresst/control/ack')

In [ ]:
client.on_message = receive_and_print

In [7]:
# payload = {
#     "ChannelID": 1,
#     "DAC": 1.6,  # normed to [-1,1]
#     "BiasCurrent": 2.4,  # normed to [-1,1]
# }

# result = client.publish('ccscresst/control/set', json.dumps(payload))
# check(result)

In [8]:
# channel_info = {"SubscribeToChannel": [6]}
# result = client.publish('ccscresst/subscription/set', json.dumps(channel_info))
# check(result)

In [ ]:
client.loop_forever()